# Demo de gestión de atención a desastres

![texto alternativo](https://upload.wikimedia.org/wikipedia/commons/b/b0/Parkes_castle.jpg)

Hubo un desbordamiento de un río en un municipio colombiano, y se puso a la población en albergues. Este cuaderno hace un análisis de los damnificados del albergue que se encuentra en el estadio municipal.

Por: Norman Simón Rodríguez, MSc. E-mail: nsrodriguezc@unal.edu.co (Versión: Junio 22 de 2018).

**Nota**: Los datos usados en este cuaderno son simulados, no representan personas reales.

Vamos a importar las librerías necesarias:

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Y vamos a importar los datos del censo del albergue del estadio.

In [0]:
url_censo = 'https://raw.githubusercontent.com/politicasydatos/curso_intensivo_estadistica/master/datos/censo_albergue_estadio.csv'
censo_estadio = pd.read_csv(url_censo)

## Explorando los datos

Miremos la proporción de hombres y mujeres, y también la distribución etaria:

In [0]:
# ¿Cuánta gente hay en el albergue?
print("En el albergue hay {} personas.".format(str(len(censo_estadio))))

# La tabla de frecuencias absolutas
pd.crosstab(index=censo_estadio["sexo"],columns="count")

In [0]:
# El histograma de edades
plt.hist(censo_estadio["edad"].astype(int))
plt.title("Histograma de edades")
plt.xlabel("Intervalos de edades")
plt.ylabel("Frecuencia absoluta")
plt.show()


Podemos sacar algunas estadísticas interesantes. Por ejemplo, las pérdidas totales:

In [0]:
sum(censo_estadio['perdidas_estimadas'].dropna().astype(int))

O el ingreso total y per cápita:

In [0]:
# El ingreso total
Y = sum(censo_estadio['ingresos_mensuales'].astype(int))
print("El ingreso total de la comunidad albergada es {} pesos por mes.".format(str(Y)))

# El ingreso per cápita
y = Y / len(censo_estadio)
y = round(y)
print("El ingreso per cápita de la comunidad albergada es {} pesos por mes por persona.".format(str(y)))

Sin embargo, no todas las personas tienen el mismo nivel de ingresos. Los ingresos se miden mejor por unidad de gasto (familia). Basándonos en los apellidos de las personas, determinemos cuáles son las familias albergadas en el estadio, y cuántos miembros tiene cada una de ellas.

In [0]:
tam_familias = censo_estadio.groupby('apellido')['documento_identidad'].count() # Contamos cuántas personas ("cédulas") hay por cada familia.
tam_familias = tam_familias.to_frame()
#print(tam_familias.index.name)
tam_familias.columns = ['tamano_familia']
print("En el albergue hay {} familias.".format(str(len(tam_familias))))
tam_familias

Ahora para cada familia averigüemos su ingreso total mensual y juntemos los datos con lo que ya sabemos de cada una:

In [0]:
censo_estadio['ingresos_mensuales']= pd.to_numeric(censo_estadio['ingresos_mensuales'])
familias_Y = censo_estadio.groupby('apellido')['ingresos_mensuales'].sum()
familias_Y = familias_Y.to_frame() # Son Series, hay que convertirlas a DataFrames
familias_Y.columns = ['ingreso_total_familia']
familias = tam_familias.join(familias_Y)
del familias_Y
del tam_familias
familias

## Datos para la toma de decisiones

El precio de una canasta básica depende de la cantidad de niños, adultos y adultos mayores que haya en la familia. Se sabe que por cada niño se gastan 80 mil pesos al mes, por cada adulto 70 mil. Sabiendo el ingreso total actual de cada familia, ahora calculamos el costo de la canasta familiar para cada familia, lo cual nos permite saber el subsidio requerido por familia (el subsidio puede ser en dinero o en especie).

¿Cuánto debería ser el subsidio por familia?

Para averiguarlo, creamos una función para calcular el gasto mínimo de cada persona.

In [0]:
def calcular_gasto_min_individual(edad):
    gasto_nino = 80000
    gasto_adulto = 70000
    if edad < 18: # Si es un niño
        return gasto_nino
    else: # Si es un adulto
        return gasto_adulto

Y calculamos el gasto mínimo de cada persona en el albergue:

In [0]:
censo_estadio['gasto_min_individual'] = censo_estadio['edad'].astype(int).apply(calcular_gasto_min_individual)
censo_estadio.head()
#censo_estadio[censo_estadio['edad'].astype(int) < 18].head()

Y ahora se usa esa función para calcular, para cada familia, la canasta mínima.

In [0]:
familias_C = censo_estadio.groupby('apellido')['gasto_min_individual'].sum()
familias_C = familias_C.to_frame() # Son Series, hay que convertirlas a DataFrames
familias_C.columns = ['canasta_min_familia']
familias_C = familias_C.join(familias)
familias_C

Ahora simplemente calculamos la diferencia entre lo que la familia necesita (la canasta) y lo que la familia tiene (ingreso total). Esa diferencia nos dice cuánto subsidio necesita cada familia.

In [0]:
familias_C['subsidio'] = familias_C['canasta_min_familia'] - familias_C['ingreso_total_familia']
familias_C

Los subsidios negativos representan familias que ganan más de lo que necesitan. En esos casos no es necesario darles subsidio. Eliminando los subsidios negativos, el subsidio total por el albergue es de:

In [0]:
datos_filtrados = familias_C['subsidio'][familias_C['subsidio'] >= 0]
subsidio_total_estadio = sum(datos_filtrados)
print("En total se necesitan {} pesos para dar en subsidios.".format(str(subsidio_total_estadio)))

Exportamos los datos más relevantes:

In [0]:
from google.colab import files

familias_C.to_csv('datos_familias.csv')

files.download('datos_familias.csv')

## Datos adicionales

Podemos mirar cuántas oleadas de evacuación ha habido, y si coinciden con los días en los que se hicieron las alertas.

In [0]:
evacuaciones = pd.crosstab(index=censo_estadio['fecha_evacuacion'],columns="count")
print(evacuaciones)
plt.bar(x=evacuaciones.index, height=evacuaciones['count'])
plt.title('Personas evacuadas por día')
plt.show()

Y también podemos mirar cuáles familias necesitan atención médica prioritaria por casos de EDA:

In [0]:
censo_estadio['diarrea'] = pd.to_numeric(censo_estadio['diarrea'])
EDA = censo_estadio.groupby('apellido')['diarrea'].sum()
EDA = EDA.to_frame() # Son Series, hay que convertirlas a DataFrames
EDA.columns = ['casos_de_eda']
EDA.sort_values('casos_de_eda', ascending=False)

## Otros recursos

El blog del BID relata la experiencia exitosa de Tabasco (México), en la atención a las inundaciones de 2009 [(link)](https://blogs.iadb.org/abierto-al-publico/2016/02/25/como-el-big-data-ayuda-en-la-gestion-de-desastres-naturales-el-caso-de-tabasco-en-2009/?utm_source=Newsletter31&utm_medium=EmailKNM&utm_campaign=Blog&utm_term=Blog&utm_content=Junio) y también el uso del "Crowd" para [recolectar datos](https://blogs.iadb.org/abierto-al-publico/2018/06/07/apoyar-a-guatemala/?utm_source=Newsletter31&utm_medium=EmailKNM&utm_campaign=Blog&utm_term=Blog&utm_content=Junio) en el desastre del Volcán de Fuego. En el caso de Tabasco, resalta que se mezclaron datos de las autoridades, datos satelitales y datos de llamadas por celular para determinar qué tanta movilidad había después de cada comunicado de alerta. Se determinó que cada vez que llovía fuertemente se emitía la alerta, pero la gente no se movía sino sólo cuando se presentaba un desastre. En Guatemala se está usando crowdsourcing para triangular geográficamente los albergues informales.